In [68]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, models

%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd



import math
import time
import os
print(os.listdir('./data'))
file_path ='./data'

['sample_submission.csv', 'test.csv', 'train.csv']


In [69]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
batch_size = 128

In [70]:
train = pd.read_csv(file_path + "/train.csv")
test = pd.read_csv(file_path + "/test.csv")
submit = pd.read_csv(file_path + "/sample_submission.csv")

# custom dataset

In [71]:
class Dataset(object):
    
    def __len__(self):
        raise NotImplementedError
    
    def __getitem__(self, index):
        raise NotImplementedError
    
    def __add__(self, other):
        return ConcatDataset([self, other])

In [72]:
class DatasetMNIST2(Dataset):
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((28, 28, 1))
        label = self.data.iloc[index, 0]
        
        if self.transform is not None :
            image = self.transform(image)
        
        return image, label

In [73]:
trainset = DatasetMNIST2('./data/train.csv', transform=transforms.ToTensor())

In [74]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4, shuffle=True)

In [75]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)
print(images[1].shape)
print(labels[1].item())

torch.Size([4, 1, 28, 28])
torch.Size([1, 28, 28])
7


In [76]:
img = images[1]
print(type(img))

<class 'torch.Tensor'>


In [77]:
npimg = img.numpy()
print(npimg.shape)

(1, 28, 28)


In [78]:
npimg = np.transpose(npimg, (1, 2, 0))
print(npimg.shape)

(28, 28, 1)


# Network

In [79]:
class LeNet(nn.Module):
    
    def __init__(self): 
        
        super(LeNet, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),         # (N, 1, 28, 28) -> (N,  6, 24, 24)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),               # (N, 6, 24, 24) -> (N,  6, 12, 12)
            nn.Conv2d(6, 16, 5),                    # (N, 6, 12, 12) -> (N, 16, 8, 8)  
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(2, stride=2)                # (N,16, 8, 8) -> (N, 16, 4, 4)
        )
        self.classifier = nn.Sequential(
            nn.Linear(256,120),                     # (N, 256) -> (N, 120)
            nn.ReLU(inplace=True),
            nn.Linear(120,84),                      #  (N, 120) -> (N, 84)
            nn.ReLU(inplace=True),
            nn.Linear(84,10)                       # (N, 84)  -> (N, 10)
        )
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def predict(self, X):
        Y_pred = self.forward(X)
        return Y_pred

In [ ]:
def training(network, criterion, optimizer, epoch_num, test=True):
    
    print("Start Training with", device, epoch_num, "overall epoch")
    network.to(device)
    """
    Create Dataloader for training and validating
    """
    composed_transform = transforms.Compose([Regularize(), ToTensor()])
    digit_dataset = DigitDataset('train.csv', '../input/', train=True, transform=composed_transform)
    if test:
        train_indices, val_indices = train_validate_split(digit_dataset.digit_df)
        train_sampler = sampler.SubsetRandomSampler(train_indices)
        val_sampler = sampler.SubsetRandomSampler(val_indices)
        train_dataloader = DataLoader(
            digit_dataset,
            batch_size=32,
            shuffle=False,
            sampler=train_sampler,
            num_workers=4,
            pin_memory=True
        )
        val_dataloader = DataLoader(
            digit_dataset,
            batch_size=32,
            shuffle=False,
            sampler=val_sampler,
            num_workers=4,
            pin_memory=True
        )
        print("Training with validation, ", "Overall Data:", len(train_indices)+len(val_indices))
        print("Training Data:", len(train_indices), "Validate Data:", len(val_indices))
    else:
        train_dataloader = DataLoader(
            digit_dataset,
            batch_size=32,
            shuffle=True,
            num_workers=4,
            pin_memory=True
        )
        val_dataloader = None
        print("Training all data, ", "Overall Data:", len(digit_dataset))
    """
    Start Training
    """
    batch_num = 0
    ita = []
    loss_avg = []
    val_acc = []
    for epoch in range(epoch_num):
        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            digits, labels = data
            digits, labels = digits.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = network(digits)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            batch_num += 1
            if test == True and i % 500 == 499:
                ita.append(batch_num)
                loss_avg.append(running_loss/500.)
                val_acc.append(validating(network, val_dataloader))
                running_loss = 0.
    if test:
        train_accuracy = validating(network, train_dataloader)
        val_accuracy = validating(network, val_dataloader)
        print('Training accuracy: %.5f' % (train_accuracy))
        print('Validation accuracy: %.5f' % (val_accuracy))
    return network, ita, loss_avg, val_acc

In [44]:
net = LeNet()
out = net(images)

In [45]:
print(out)

tensor([[-0.0479,  0.0611,  0.1173,  0.0203, -0.0228, -0.0456,  0.1334, -0.0707,
         -0.0292,  0.0015],
        [-0.0425,  0.0596,  0.1182,  0.0231, -0.0256, -0.0435,  0.1291, -0.0687,
         -0.0257, -0.0037],
        [-0.0447,  0.0615,  0.1163,  0.0250, -0.0277, -0.0453,  0.1341, -0.0655,
         -0.0281, -0.0026],
        [-0.0450,  0.0601,  0.1146,  0.0213, -0.0286, -0.0433,  0.1353, -0.0684,
         -0.0293, -0.0037]], grad_fn=<AddmmBackward>)


In [48]:
def evaluation(dataloader):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [49]:
net = LeNet().to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters())

In [50]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

In [51]:
max_epochs = 16

for epoch in range(max_epochs):

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        
    print('Epoch: %d/%d, Train acc: %0.2f' % (epoch, max_epochs, evaluation(trainloader)))

Epoch: 0/16, Train acc: 90.31
Epoch: 1/16, Train acc: 93.73
Epoch: 2/16, Train acc: 95.99
Epoch: 3/16, Train acc: 96.96
Epoch: 4/16, Train acc: 97.42
Epoch: 5/16, Train acc: 97.91
Epoch: 6/16, Train acc: 98.42
Epoch: 7/16, Train acc: 98.58
Epoch: 8/16, Train acc: 98.93
Epoch: 9/16, Train acc: 98.65
Epoch: 10/16, Train acc: 98.76
Epoch: 11/16, Train acc: 99.10
Epoch: 12/16, Train acc: 99.29
Epoch: 13/16, Train acc: 99.36
Epoch: 14/16, Train acc: 99.28
Epoch: 15/16, Train acc: 99.13


# Preparing Test Data

In [52]:
testset = np.loadtxt('./data/test.csv',skiprows=1,delimiter=",")

In [53]:
testset = testset.reshape(testset.shape[0],28,28,1)

In [54]:
testset = testset/255

In [55]:
testset.shape

(28000, 28, 28, 1)

In [56]:
testset = np.transpose(testset, (0, 3, 1,2))

In [57]:
testset.shape

(28000, 1, 28, 28)

In [62]:
testset = torch.tensor(testset)

In [63]:
testset = testset.float()

In [64]:
testset = testset.to(device)

In [65]:
Y_pred_val = net.predict(testset)
pred = torch.argmax(Y_pred_val, dim=1)
pred.shape

torch.Size([28000])

In [66]:
pred = pred.to('cpu')
results = pd.Series(pred , name = 'Label')
submission = pd.concat([pd.Series(range(1,28001),name = 'ImageId'),results],axis=1)
submission.to_csv("final_submission.csv",index=False)

# Reference
- https://www.kaggle.com/tauseef6462/cnn-lenet-architecture-using-pytorch
- https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
- https://www.kaggle.com/ankschoubey/lenet5-based-cnn-with-98-94-using-pytorch
- https://www.kaggle.com/miketonson1006/pytorch-from-a-beginner-s-view